In [1]:
# State

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path
import itertools

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)
from quara.objects.operators import tensor_product

from quara.data_analysis.simulation import SimulationSetting

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys_1 = ElementalSystem(0, get_normalized_pauli_basis())
c_sys_1 = CompositeSystem([e_sys_1])

e_sys_2 = ElementalSystem(1, get_normalized_pauli_basis())
c_sys_2 = CompositeSystem([e_sys_2])

tester_objects = []

func_list = [get_x_measurement, get_y_measurement, get_z_measurement]

for i, funcs in enumerate(itertools.product(func_list, func_list)):
    povm1 = funcs[0](c_sys_1)
    povm2 = funcs[1](c_sys_2)
    
    povm_2qubit = tensor_product(povm1, povm2)
    tester_objects.append(povm_2qubit)

In [6]:
# |0><0|
true_object = tensor_product(get_z0_1q(c_sys_1), get_z0_1q(c_sys_2))

In [7]:
true_object.calc_eigenvalues()

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [8]:
num_data = [100, 1000, 10000]
# n_rep = 100
n_rep = 2

case_name_list = [
    "Linear(True)",
    "Linear(False)",
    "ProjectedLinear(True)",
    "ProjectedLinear(False)"
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed)]
para_list = [
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    start = time.time()

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=n_rep,
    )

    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator
    )
    print(simulation_setting)
    simulation_settings.append(simulation_setting)

    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time: {0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/2 [00:00<?, ?it/s]Name: Linear(True)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.008116650581359863[min]

  0%|          | 0/2 [00:00<?, ?it/s]Name: Linear(False)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.009400002161661784[min]

  0%|          | 0/2 [00:00<?, ?it/s]Name: ProjectedLinear(True)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.05301666259765625[min]

100%|██████████| 2/2 [00:05<00:00,  2.90s/it]Name: ProjectedLinear(False)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.09665005207061768[min]




In [9]:
report.export_report("qst_sample_2qubit.pdf",
    estimation_results_list=estimation_results_list,   # 「EstimationResultのリスト」のリスト
    simulation_settings=simulation_settings,
    true_object=true_object,  # True Object
    tester_objects = tester_objects,   # Tester Objectのリスト.
    seed=seed,  # 推定で使ったseed（オプション）
    computation_time=sum(elapsed_times) # 処理時間の合計（オプション）
)

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...
100%|██████████| 2/2 [00:00<00:00, 1000.31it/s]
​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  Linear(True)=[0.026266666666666962, 0.0030808888888888496, 0.00010473333333334343]
  Linear(False)=[0.021099999999999963, 0.0024891111111111155, 0.000287905555555558]
  ProjectedLinear(True)=[0.01382077211372456, 0.0030640260515698856, 0.00029379182885952773]
  ProjectedLinear(False)=[0.010107085051013744, 0.0008938435069973268, 0.0002987780056118095]
{'make_graphs_func': <function make_mses_graphs_estimator at 0x000001EDBB724B70>, 'mse_type': 'estimator', 'estimation_results_list': [[<quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x000001EDBC08AA20>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x000001EDBC08A908>],